In [1]:
import numpy as np
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')
import itertools



import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set_theme(style="white")
%matplotlib inline


from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [2]:
monthly = pd.read_csv('co2_mm_mlo.csv')

In [3]:
monthly

,year,month,decimal date,average,interpolated,trend,ndays,Unnamed: 7
0,1958,3,1958.2027,315.70,314.43,-1,-9.99,-0.99
1,1958,4,1958.2877,317.45,315.16,-1,-9.99,-0.99
2,1958,5,1958.3699,317.51,314.71,-1,-9.99,-0.99
3,1958,6,1958.4548,317.24,315.14,-1,-9.99,-0.99
4,1958,7,1958.5370,315.86,315.18,-1,-9.99,-0.99
...,...,...,...,...,...,...,...,...
751,2020,10,2020.7917,411.51,414.92,30,0.22,0.08
752,2020,11,2020.8750,413.11,415.14,27,0.80,0.29
753,2020,12,2020.9583,414.25,415.00,30,0.48,0.17
754,2021,1,2021.0417,415.52,415.26,29,0.44,0.16


In [4]:
weekly = pd.read_csv('co2_weekly_mlo.csv')

In [5]:
weekly

,year,month,day,decimal,average,ndays,1 year ago,10 years ago,increase since 1800
0,1974,5,19,1974.3795,333.37,5,-999.99,-999.99,50.40
1,1974,5,26,1974.3986,332.95,6,-999.99,-999.99,50.06
2,1974,6,2,1974.4178,332.35,5,-999.99,-999.99,49.60
3,1974,6,9,1974.4370,332.20,7,-999.99,-999.99,49.65
4,1974,6,16,1974.4562,332.37,7,-999.99,-999.99,50.06
...,...,...,...,...,...,...,...,...,...
2436,2021,1,24,2021.0644,416.01,7,413.96,391.68,135.50
2437,2021,1,31,2021.0836,417.12,7,414.50,392.19,136.49
2438,2021,2,7,2021.1027,416.92,7,414.91,391.64,136.18
2439,2021,2,14,2021.1219,416.51,7,414.12,391.92,135.67


In [6]:
weekly.isna().sum()

year                   0
month                  0
day                    0
decimal                0
average                0
ndays                  0
1 year ago             0
10 years ago           0
increase since 1800    0
dtype: int64

In [7]:
weekly['Date'] = pd.to_datetime(weekly[['year', 'month', 'day']])

In [22]:
weekly

,year,month,day,decimal,average,ndays,1 year ago,10 years ago,increase since 1800,Date
0,1974,5,19,1974.3795,333.37,5,-999.99,-999.99,50.40,1974-05-19
1,1974,5,26,1974.3986,332.95,6,-999.99,-999.99,50.06,1974-05-26
2,1974,6,2,1974.4178,332.35,5,-999.99,-999.99,49.60,1974-06-02
3,1974,6,9,1974.4370,332.20,7,-999.99,-999.99,49.65,1974-06-09
4,1974,6,16,1974.4562,332.37,7,-999.99,-999.99,50.06,1974-06-16
...,...,...,...,...,...,...,...,...,...,...
2436,2021,1,24,2021.0644,416.01,7,413.96,391.68,135.50,2021-01-24
2437,2021,1,31,2021.0836,417.12,7,414.50,392.19,136.49,2021-01-31
2438,2021,2,7,2021.1027,416.92,7,414.91,391.64,136.18,2021-02-07
2439,2021,2,14,2021.1219,416.51,7,414.12,391.92,135.67,2021-02-14


In [10]:
weekly.column

Index(['year', 'month', 'day', 'decimal', 'average', 'ndays', '1 year ago',
       '10 years ago', 'increase since 1800', 'Date'],
      dtype='object')

In [26]:
years = weekly.loc[weekly['average'] == -999.99].index.values

In [28]:
years

array([  72,   81,   82,   83,   84,  110,  409,  412,  413,  481,  515,
        516,  517,  518, 1639, 1780, 1781, 1782])

In [29]:
for index in years:
    x1 = weekly.loc[index - 1, 'average']
    x2 = weekly.loc[index + 1, 'average']
    weekly.loc[index, 'average'] = (x1 + x2) / 2

In [31]:
weekly['average'].value_counts()

393.73    3
353.99    3
354.94    3
376.41    3
372.04    3
         ..
358.38    1
391.39    1
342.33    1
399.40    1
334.00    1
Name: average, Length: 2145, dtype: int64

In [ ]:
co2_weekly = weekly.copy()

In [ ]:
co2_weekly = co2_weekly[co2_weekly.average != -999.99]

In [ ]:
ppm_mean = co2_weekly.average.mean()

In [ ]:
weekly['average'] = weekly['average'].replace([-999.99], [ppm_mean])

In [ ]:
weekly[weekly]

In [ ]:
season = weekly
season['Date'] = weekly.Date
season['Year'] = weekly['Date'].dt.year
season['Month'] = weekly['Date'].dt.month
spivot = pd.pivot_table(season, index='Month', columns = 'Year', values = 'average', aggfunc=np.mean)
spivot.plot(figsize=(20,10), linewidth=3)
plt.legend(loc = 'best')
plt.show()

In [ ]:
weekly.drop(columns = ['year', 'month', 'day'], inplace = True)

In [ ]:
weekly1.set_index('Date', inplace = True)

In [ ]:
avg_week = weekly.copy()

In [ ]:
avg_week.columns

In [ ]:
avg_week.drop(columns = ['decimal', 'ndays', '1 year ago', '10 years ago', 'increase since 1800'], inplace =True)

In [ ]:
avg_week.value_counts()

In [ ]:
avg_week['average'].mean()

In [ ]:
avg_week = avg_week.mask(avg_week == -999.99).fillna(0)

In [ ]:
avg_week.plot(ylabel = 'ppm')

In [ ]:
month = avg_week.resample('M').mean()
month_std = avg_week.resample('M').std()


In [ ]:
fig = plt.figure(figsize = (20,10))
plt.plot(month, color = 'tomato', label = 'Average Monthly CO2')
plt.plot(month_std, color = 'gold', label = 'STD Monthly CO2')
plt.legend(loc = 'best')
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller
test = adfuller(avg_week['average'][1:-1])
dfoutput = pd.Series(test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
print(dfoutput)

In [ ]:
plt.style.use('classic')

In [ ]:
plot_acf(avg_week[:-1], alpha=0.05);

In [ ]:
plot_pacf(avg_week[:-1], alpha = 0.001, lags=20);